In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from implementations_JB import *
from proj1_helpers import *
%load_ext autoreload
%autoreload 2

In [28]:
def ultimate_split(yb, input_data, ids):
    """
    The ultimate split. THE real BADA$$.
    This is some real shit
    """
    
    mask_0 = (input_data[:,22] == 0)
    mask_1 = (input_data[:,22] == 1)
    mask_2 = (input_data[:,22] == 2)
    mask_3 = (input_data[:,22] == 3)

    
    return yb[mask_0], yb[mask_1], yb[mask_2], yb[mask_3], \
input_data[mask_0], input_data[mask_1], input_data[mask_2], input_data[mask_3], \
ids[mask_0], ids[mask_1], ids[mask_2], ids[mask_3]

In [51]:
def remove_features_0(input_data_0):
    """
    Remove the useless features if the jet num feature is equal to 0
    """
    useless_features_index = [4, 5, 6, 12, 23, 24, 25 , 26, 27, 28]# , 30, 33]
    
    mask = np.ones(int(input_data_0.shape[1]), dtype=bool)
    mask[(useless_features_index)] = False
    return input_data_0[:,mask]

def remove_features_1(input_data_1):
    """
    Remove the useless features if the jet num feature is equal to 1
    """
    useless_features_index = [4, 5, 6, 26, 27, 28]#, 30, 33]
    
    mask = np.ones(int(input_data_1.shape[1]), dtype=bool)
    mask[(useless_features_index)] = False
    return input_data_1[:,mask]

    # No need for jet_num equal to 2 or 3 : all the features are kept   

In [6]:
def angle_features_processing(data_path):
    """
    We want to change all the features representing an angle into two features
    one for cosinus of the angle and another for sinus of the angle.
    """
    
    yb, input_data, ids = load_csv_data(data_path,sub_sample = False)

    angle_features = [15, 18, 20, 25, 28]
    
    new_data = np.zeros((input_data.shape[0], input_data.shape[1] + len(angle_features) ))
    
    for k in range(input_data.shape[1]):
        if k not in angle_features:
            new_data[:, k] = input_data[:, k]
        
    for idx, column in enumerate(angle_features): 
        
        new_data[:, column] = np.cos(input_data[:, column])
        new_data[:, input_data.shape[1] + idx] = np.sin(input_data[:, column])
    
    return yb, new_data, ids

In [8]:
def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    poly = np.ones((len(x), 1))
    for deg in range(1, degree+1):
        poly = np.c_[poly, np.power(x, deg)]
    return poly

In [46]:
def least_squares_implementation(yb, data, ids, max_deg):
    """
    The inputs of this function are arrays containing the data from the separation into 4 parts
    depending on the value of the jet_num index
    """
    
    best_deg = np.zeros(4)
    min_loss = np.Inf*np.ones(4)
    w = np.zeros((4, max_deg+1))
    losses = np.zeros((4, max_deg+1))
    
    for i in range(3):
        # We test for polynomes from degree 0 to max_deg
        for deg in range(max_deg + 1):

            poly = build_poly(data[i], deg)
            w[i, deg], losses[i, deg] = least_squares(yb[i], poly)
            
            if (losses[i, deg] < min_loss[i]):
                min_loss[i] = losses[i, deg]
                best_deg[i] = deg
            
    return w[0, best_deg[0]], w[1, best_deg[1]], w[2, best_deg[2]], w[3, best_deg[3]] 

In [52]:
# Processing the angle features
# yb, input_data, ids =  angle_features_processing("train.csv")
yb, input_data, ids = load_csv_data("train.csv",sub_sample = False)


yb0, yb1, yb2, yb3, data0, data1, data2, data3, ids0, ids1, ids2, ids3 = ultimate_split(yb, input_data, ids)
input_data = standardize(input_data)


data0 = remove_features_0(data0)
data1 = remove_features_1(data1)

In [53]:
yb = np.array([yb0,yb1,yb2,yb3])
data = np.array([data0,data1,data2,data3])
ids = np.array([ids0,ids1,ids2,ids3])

w0, w1, w2, w3 = least_squares_implementation(yb, data, ids, 10)

LinAlgError: Singular matrix